# WEEK 4 Exercises - Making Queries

![ERD-E-COMMERCE](https://github.com/it-web-pro/django-week4/blob/main/images/WEEK3-ERD(e-commerce).png?raw=true)

**ให้นักศึกษาสร้างโปรเจคใหม่ใหม่ชื่อ`myshop`**

**จากนั้นให้ทำการ startapp ใหม่ชื่อ `shop`**

**ให้นักศึกษาทำการเพิ่ม model นี้ในไฟล์ shop/models.py**

In [ ]:
from django.db import models

# Create your models here.


class Customer(models.Model):
    first_name = models.CharField(max_length=150)
    last_name = models.CharField(max_length=200)
    email = models.CharField(max_length=150)
    address = models.JSONField(null=True)

class ProductCategory(models.Model):
    name = models.CharField(max_length=150)

class Product(models.Model):
    name = models.CharField(max_length=150)
    description = models.TextField(null=True, blank=True)
    remaining_amount = models.PositiveIntegerField(default=0)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    categories = models.ManyToManyField(ProductCategory)

class Cart(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    create_date = models.DateTimeField()
    expired_in = models.PositiveIntegerField(default=60)
    
class CartItem(models.Model):
    cart = models.ForeignKey(Cart, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Order(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    order_date = models.DateField()
    remark = models.TextField(null=True, blank=True)

class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Payment(models.Model):
    order = models.OneToOneField(Order, on_delete=models.PROTECT)
    payment_date = models.DateField()
    remark = models.TextField(null=True, blank=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)

class PaymentItem(models.Model):
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    order_item = models.OneToOneField(OrderItem, on_delete=models.CASCADE)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)
    
class PaymentMethod(models.Model):
    class MethodChoices(models.Choices):
        QR = "QR"
        CREDIT = "CREDIT"
    
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    method = models.CharField(max_length=15, choices=MethodChoices.choices)
    price = models.DecimalField(max_digits=10, decimal_places=2)

**จากนั้นให้ทำการ migrate และ run คำสั่งในไฟล์ shop.sql**

In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

### 1. ให้นักศึกษา Query ค้นหาข้อมูลมาแสดงให้ถูกต้องตามโจทย์

1.1 query หาข้อมูล `Order` ทั้งหมดที่เกิดขึ้นในเดือน `พฤษภาคม` และ`ราคา`ของ Order มาแสดงผล 10 รายการแรก และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
ORDER ID:22, DATE: 2024-05-01, PRICE: 4890.00
ORDER ID:23, DATE: 2024-05-01, PRICE: 2540.00
ORDER ID:24, DATE: 2024-05-01, PRICE: 1720.00
ORDER ID:25, DATE: 2024-05-02, PRICE: 322499.00
ORDER ID:26, DATE: 2024-05-02, PRICE: 3399.00
ORDER ID:27, DATE: 2024-05-02, PRICE: 1190.00
ORDER ID:28, DATE: 2024-05-03, PRICE: 9499.00
ORDER ID:29, DATE: 2024-05-03, PRICE: 700.00
ORDER ID:30, DATE: 2024-05-03, PRICE: 1690.00
ORDER ID:31, DATE: 2024-05-04, PRICE: 3280.00
```

In [14]:
from shop.models import *
for order in Order.objects.filter(order_date__month = 5, payment__price__gt=1000)[:10]:
    print(f"ORDER ID:{order.id}, DATE: {order.order_date}")

ORDER ID:22, DATE: 2024-05-01
ORDER ID:23, DATE: 2024-05-01
ORDER ID:24, DATE: 2024-05-01
ORDER ID:25, DATE: 2024-05-02
ORDER ID:26, DATE: 2024-05-02
ORDER ID:27, DATE: 2024-05-02
ORDER ID:28, DATE: 2024-05-03
ORDER ID:30, DATE: 2024-05-03
ORDER ID:31, DATE: 2024-05-04
ORDER ID:32, DATE: 2024-05-04


1.2 query หาข้อมูล `Product` ที่มีคำลงท้ายว่า `features.` ในรายละเอียดสินค้า และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT ID: 7, DESCRIPTION: High-resolution digital camera with advanced photography features.
PRODUCT ID: 10, DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT ID: 14, DESCRIPTION: Split air conditioner with remote control and energy-saving features.
PRODUCT ID: 45, DESCRIPTION: Customizable racing track set with loop and jump features.
```

In [66]:
for product in Product.objects.exclude(description__endswith = "features."):
    print(f"PRODUCT ID: {product.id}, DESCRIPTION: {product.description}")

PRODUCT ID: 2, DESCRIPTION: A lightweight and high-performance laptop for professionals.
PRODUCT ID: 3, DESCRIPTION: An ultra-HD smart television with streaming capabilities.
PRODUCT ID: 4, DESCRIPTION: Wireless earphones with noise-canceling technology.
PRODUCT ID: 5, DESCRIPTION: A compact tablet for entertainment and productivity.
PRODUCT ID: 6, DESCRIPTION: A powerful gaming console with immersive graphics.
PRODUCT ID: 8, DESCRIPTION: High-speed wireless router for home or office use.
PRODUCT ID: 9, DESCRIPTION: Compact and portable power bank for charging devices on the go.
PRODUCT ID: 11, DESCRIPTION: Energy-efficient refrigerator with ample storage space.
PRODUCT ID: 12, DESCRIPTION: Front-loading washing machine with multiple wash programs.
PRODUCT ID: 13, DESCRIPTION: Compact microwave oven with defrost and cooking functions.
PRODUCT ID: 15, DESCRIPTION: Powerful vacuum cleaner for efficient cleaning.
PRODUCT ID: 16, DESCRIPTION: Fast-boiling electric kettle with auto shut-off

1.3 query หาข้อมูล `Product` ที่มีราคาสินค้าตั้งแต่ `5000.00` ขึ้นไป และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 6, NAME: Gaming Console, PRICE: 5000.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00
PRODUCT ID: 11, NAME: Refrigerator, PRICE: 9000.00
PRODUCT ID: 14, NAME: Air Conditioner, PRICE: 18900.00
PRODUCT ID: 31, NAME: Sofa, PRICE: 7000.00
PRODUCT ID: 54, NAME: Automatic Pet Feeder, PRICE: 7900.00
PRODUCT ID: 61, NAME: Diamond Stud Earrings, PRICE: 320000.00
PRODUCT ID: 62, NAME: Silver Charm Bracelet, PRICE: 70000.00
PRODUCT ID: 63, NAME: Gold Pendant Necklace, PRICE: 59000.00
PRODUCT ID: 64, NAME: Gemstone Ring, PRICE: 9000.00
PRODUCT ID: 65, NAME: Rose Gold Hoop Earrings, PRICE: 1200000.00
```

In [68]:
for product in Product.objects.filter(price__lt=5000.00):
    print(f"PRODUCT ID: {product.id}, NAME: {product.name}, PRICE: {product.price}")

PRODUCT ID: 4, NAME: Bluetooth Earphones, PRICE: 350.00
PRODUCT ID: 8, NAME: Wireless Router, PRICE: 990.00
PRODUCT ID: 9, NAME: Portable Power Bank, PRICE: 390.00
PRODUCT ID: 10, NAME: Smartwatch, PRICE: 4500.00
PRODUCT ID: 12, NAME: Washing Machine, PRICE: 3900.00
PRODUCT ID: 13, NAME: Microwave Oven, PRICE: 1290.00
PRODUCT ID: 15, NAME: Vacuum Cleaner, PRICE: 4000.00
PRODUCT ID: 16, NAME: Electric Kettle, PRICE: 2499.00
PRODUCT ID: 17, NAME: Toaster, PRICE: 390.00
PRODUCT ID: 18, NAME: Electric Fan, PRICE: 290.00
PRODUCT ID: 19, NAME: Rice Cooker, PRICE: 690.00
PRODUCT ID: 20, NAME: Electric Iron, PRICE: 1200.00
PRODUCT ID: 21, NAME: Men's T-Shirt, PRICE: 200.00
PRODUCT ID: 22, NAME: Women's Jeans, PRICE: 450.00
PRODUCT ID: 23, NAME: Unisex Hoodie, PRICE: 239.00
PRODUCT ID: 24, NAME: Women's Dress, PRICE: 3200.00
PRODUCT ID: 25, NAME: Men's Formal Shirt, PRICE: 690.00
PRODUCT ID: 26, NAME: Women's Athletic Shorts, PRICE: 700.00
PRODUCT ID: 27, NAME: Men's Polo Shirt, PRICE: 450.00
P

1.4 query หาข้อมูล `Product` ที่มีราคาสินค้าน้อยกว่า `200.00` และมากกว่า `100.00` และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
```

In [22]:
for product in Product.objects.filter(price__range=(100.01, 199.99)):
    print(f"PRODUCT ID: {product.id}, NAME: {product.name}, PRICE: {product.price}")

PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
PRODUCT ID: 66, NAME: Salvage the Bones, PRICE: 129.00


### 2. เพิ่ม ลบ แก้ไข สินค้า

#### หมวดหมู่สินค้า
- Information technology
- Electronics
- Clothing and Apparel
- Home Appliances
- Furniture
- Toys and Games
- Books and Media
- Pet Supplies
- Jewelry

{'id': 1, 'name': 'Information Technology'}, 
{'id': 2, 'name': 'Electronics'}, 
{'id': 3, 'name': 'Clothing and Apparel'}, 
{'id': 4, 'name': 'Home Appliances'}, 
{'id': 5, 'name': 'Furniture'}, 
{'id': 6, 'name': 'Toys and Games'}, 
{'id': 7, 'name': 'Books and Media'}, 
{'id': 8, 'name': 'Pet Supplies'}, 
{'id': 9, 'name': 'Jewelry'}

2.1 ให้เพิ่มสินค้าใหม่จำนวน 3 รายการ (0.5 คะแนน)

```txt
สินค้าที่ 1
ชื่อ: Philosopher's Stone (1997)
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 20
รายละเอียดซ: By J. K. Rowling.
ราคา: 790

สินค้าที่ 2
ชื่อ: Me Before You
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 40
รายละเอียดซ: A romance novel written by Jojo
ราคา: 390

สินค้าที่ 3
ชื่อ: Notebook HP Pavilion Silver
หมวดหมู่สินค้า: Information Technology และ Electronics
จำนวนคงเหลือ: 10
รายละเอียดซ: Display Screen. 16.0
ราคา: 20000
```

In [82]:
product1 = Product(name="Philosopher's Stone (1997)", remaining_amount=20, description="By J. K. Rowling.", price=790)
product1.save()

In [83]:
# {'id': 7, 'name': 'Books and Media'}
product1_added = Product.objects.get(name="Philosopher's Stone (1997)")
cate_book_media = ProductCategory.objects.get(pk=7)
product1_added.categories.add(cate_book_media)

In [38]:
product2 = Product(name="Me Before You", remaining_amount=40, description="A romance novel written by Jojo", price=390)
# product2.save()

In [41]:
product2_added = Product.objects.get(id=68)
product2_added.categories.add(cate_book_media)

In [42]:
product3 = Product(name="Notebook HP Pavilion Silver", remaining_amount=10, description="Display Screen. 16.0s", price=2000)
product3.save()

In [43]:
# {'id': 1, 'name': 'Information Technology'}
# {'id': 2, 'name': 'Electronics'}
product3_added = Product.objects.get(pk=69)
cate_info_tech = ProductCategory.objects.get(pk=1)
cate_elec = ProductCategory.objects.get(pk=2)
product3_added.categories.add(cate_info_tech, cate_elec)

2.2 แก้ไขชื่อสินค้า จาก `Philosopher's Stone (1997)` เป็น `Half-Blood Prince (2005)` (0.5 คะแนน)

In [72]:
new_product1 = Product.objects.get(name="Philosopher's Stone (1997)") 
new_product1.name = "Half-Blood Prince (2005)"
new_product1.save()

2.3 แก้ไขชื่อหมวดหมู่สินค้า จาก `Books and Media` เป็น `Books` (0.5 คะแนน)

In [73]:
cate_book_media = ProductCategory.objects.get(name="Books")
cate_book_media.name = "Books and Media"
cate_book_media.save()

2.4 ลบสินค้าทุกตัวที่อยู่ในหมวดหมู่ `Books` (0.5 คะแนน)

In [80]:
Product.objects.filter(categories__name="Books and Media")

<QuerySet [<Product: Product object (71)>]>

In [84]:
delete_product_books = Product.objects.filter(categories__name="Books and Media")
delete_product_books.delete()

(2, {'shop.Product_categories': 1, 'shop.Product': 1})